In [1]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split
import glob
from art.attacks.evasion import FastGradientMethod, AutoAttack, FeatureAdversariesTensorFlowV2, ElasticNet,DeepFool, ProjectedGradientDescent
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist

In [2]:

damaged = glob.glob('../preprocessed_data/training/damaged/*.*')
undamaged = glob.glob('../preprocessed_data/training/undamaged/*.*')
print(damaged[:5])
print(undamaged[:5])
data = []
labels = []

for i in damaged:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (128,128, 3))
    image=np.array(image)/255.0
    data.append(image)
    labels.append(1)
for i in undamaged:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', 
    target_size= (128,128, 3))
    image=np.array(image)/255.0
    data.append(image)
    labels.append(0)


data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

X_train, X_val, ytrain, yval = train_test_split(data, labels, test_size=0.2,
                                               random_state=42)

x_test=[]
y_test=[]

test_damaged = glob.glob('../preprocessed_data/test/damaged/*.*')
test_undamaged = glob.glob('../preprocessed_data/test/undamaged/*.*')

for i in test_damaged:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (128,128))
    image=np.array(image)/255.0
    x_test.append(image)
    y_test.append(1)
for i in test_undamaged:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size= (128,128))
    image=np.array(image)/255.0
    x_test.append(image)
    y_test.append(0)

x_test = np.array(x_test)
y_test = np.array(y_test)

['../preprocessed_data/training/damaged/0692.JPEG', '../preprocessed_data/training/damaged/0826.JPEG', '../preprocessed_data/training/damaged/0411.JPEG', '../preprocessed_data/training/damaged/0575.JPEG', '../preprocessed_data/training/damaged/0909.JPEG']
['../preprocessed_data/training/undamaged/0362.jpg', '../preprocessed_data/training/undamaged/0703.jpg', '../preprocessed_data/training/undamaged/0075.jpg', '../preprocessed_data/training/undamaged/0414.jpg', '../preprocessed_data/training/undamaged/0817.JPEG']
(1840, 128, 128, 3) (1840,)


In [3]:
num_classes=2
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=32, kernel_size=1, input_shape=(128,128,3)),
  tf.keras.layers.MaxPooling2D(),
     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(filters=32, kernel_size=1, activation="relu"),
     tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation="relu"),
     tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(num_classes, activation='softmax')
])


checkpoint_filepath = './model_checkpoint/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
  optimizer=optimizer,
  loss=loss_fn,
  metrics=['accuracy'])

history=model.fit(
    X_train,
    ytrain,
    epochs=100,
    callbacks=[model_checkpoint_callback],
    validation_data= (X_val, yval), 
    shuffle=True
)

Epoch 1/100
46/46 [==============================] - 3s 37ms/step - loss: 1.6122 - accuracy: 0.6052 - val_loss: 0.8444 - val_accuracy: 0.5136
Epoch 2/100
46/46 [==============================] - 1s 19ms/step - loss: 0.5337 - accuracy: 0.7402 - val_loss: 0.8129 - val_accuracy: 0.5082
Epoch 3/100
46/46 [==============================] - 1s 19ms/step - loss: 0.3860 - accuracy: 0.8364 - val_loss: 0.8540 - val_accuracy: 0.5082
Epoch 4/100
46/46 [==============================] - 1s 19ms/step - loss: 0.3145 - accuracy: 0.8794 - val_loss: 0.9864 - val_accuracy: 0.5082
Epoch 5/100
46/46 [==============================] - 1s 19ms/step - loss: 0.2119 - accuracy: 0.9223 - val_loss: 1.1264 - val_accuracy: 0.5082
Epoch 6/100
46/46 [==============================] - 1s 19ms/step - loss: 0.1480 - accuracy: 0.9449 - val_loss: 0.9580 - val_accuracy: 0.5380
Epoch 7/100
46/46 [==============================] - 1s 19ms/step - loss: 0.1098 - accuracy: 0.9609 - val_loss: 0.8119 - val_accuracy: 0.6114
Epoch 

In [4]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
model.load_weights(checkpoint_filepath)
